# Examining problematic sulfamide geometries

In [1]:
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.drivers import get_gromacs_energies, get_openmm_energies
import multiprocessing
import tqdm
import numpy as np
import sys
import openmm
from openff.units.openmm import from_openmm
import os
# suppress stereochemistry warnings
import logging
from rdkit import Chem
import itertools

from openff.qcsubmit.results import TorsionDriveResultCollection, OptimizationResultCollection
import logging
logging.getLogger("openff").setLevel(logging.ERROR)

from openff.qcsubmit.results.filters import SMARTSFilter

import MDAnalysis as mda

In [2]:
def minimize_energy(mol,ff):
    topo = mol.to_topology()
    interchange = Interchange.from_smirnoff(force_field=ff, topology=topo,allow_nonintegral_charges=True)#,charge_from_molecules=[mol]) #,charge_from_molecules=[mol])
    integrator = openmm.VerletIntegrator(1 * openmm.unit.femtoseconds)
    simulation = interchange.to_openmm_simulation(integrator)

    # We'll store energies in two lists
    initial_energies = []
    minimized_energies = []

    # And minimized conformers in a second molecule
    minimized_molecule = Molecule.from_topology(topo)
    minimized_molecule.conformers.clear()

    conformer = mol.conformers[0]
    # Tell the OpenMM Simulation the positions of this conformer
    simulation.context.setPositions(conformer.to_openmm())

    # Keep a record of the initial energy
    initial_energies.append(
        simulation.context.getState(getEnergy=True).getPotentialEnergy()
    )

    # Perform the minimization
    simulation.minimizeEnergy()#tolerance=Quantity(value=0.0026255, unit=kilojoule/mole))#tolerance=5e-9)

    # Record minimized energy and positions
    min_state = simulation.context.getState(getEnergy=True, getPositions=True)

    minimized_energies.append(min_state.getPotentialEnergy())
    minimized_molecule.add_conformer(from_openmm(min_state.getPositions()))
    return initial_energies,minimized_energies,minimized_molecule


Choosing a few problem cases:

# Molecule 19, QCArchive ID 36971837
Bad with Sage 2.0.0, fixed in 2.1 and 2.2

Has many conformers--index 19-28. Some are distorted, some aren't

In [3]:
# test_mol1 = industry_benchmark_rm[19][1]
# test_mol1

In [4]:
# test_mol1.to_file('qcaid_36971837.sdf',file_format='sdf')

In [5]:
test_mol1=Molecule.from_file('qcaid_36971837.sdf')

In [6]:
ie_sage200_qm1,me_sage200_qm1,min_mol_sage200_qm1 = minimize_energy(test_mol1,ForceField('openff_unconstrained-2.0.0.offxml'))

In [10]:
u_sage200_min = mda.Universe(min_mol_sage200_qm1.to_rdkit())
u_sage200_min.atoms.write("sage-200_qcaid_36971837_opt.pdb")

In [7]:
ie_sage210_qm1,me_sage210_qm1,min_mol_sage210_qm1 = minimize_energy(test_mol1,ForceField('openff_unconstrained-2.1.0.offxml'))

In [9]:
u_sage210_min = mda.Universe(min_mol_sage210_qm1.to_rdkit())
u_sage210_min.atoms.write("sage-210_qcaid_36971837_opt.pdb")

In [12]:
ie_sage220_qm1,me_sage220_qm1,min_mol_sage220_qm1 = minimize_energy(test_mol1,ForceField('../../../openff_unconstrained-2.2.0.offxml',allow_cosmetic_attributes=True))

In [13]:
u_sage220_min = mda.Universe(min_mol_sage220_qm1.to_rdkit())
u_sage220_min.atoms.write("sage-220_qcaid_36971837_opt.pdb")

In [16]:
ie_sage221_qm1,me_sage221_qm1,min_mol_sage221_qm1 = minimize_energy(test_mol1,ForceField('../../../openff_unconstrained-2.2.1-rc1.offxml',allow_cosmetic_attributes=True))

In [66]:
u_sage221_min = mda.Universe(min_mol_sage221_qm1.to_rdkit())
u_sage221_min.atoms.write("sage-221_qcaid_36971837_opt.pdb")

In [70]:
u_qm_min = mda.Universe(test_mol1.to_rdkit())
u_qm_min.atoms.write("qm_qcaid_36971837_opt.pdb")

In [17]:
min_mol_sage220_qm1.to_file('qcaid_36971837_optsage220.sdf',file_format='sdf')
min_mol_sage221_qm1.to_file('qcaid_36971837_optsage221.sdf',file_format='sdf')

In [18]:
opt1_sage200 = min_mol_sage200_qm1.to_rdkit()
opt1_sage200_conf = opt1_sage200.GetConformer()

In [19]:
opt1_sage210 = min_mol_sage210_qm1.to_rdkit()
opt1_sage210_conf = opt1_sage210.GetConformer()

In [20]:
opt1_sage220 = min_mol_sage220_qm1.to_rdkit()
opt1_sage220_conf = opt1_sage220.GetConformer()

In [21]:
opt1_sage221 = min_mol_sage221_qm1.to_rdkit()
opt1_sage221_conf = opt1_sage221.GetConformer()

In [22]:
qm_rdkit1 = test_mol1.to_rdkit()
qm_conf1 = qm_rdkit1.GetConformer()

### Geometries

In [23]:
atoms1=[a for a in opt1_sage200.GetAtoms()]

In [24]:
atom_indices_s1 = [a.GetIdx() for a in atoms1 if a.GetSymbol() == 'S']

In [25]:
atom_indices_n1 = [a.GetIdx() for a in atoms1 if a.GetSymbol()=='N']

In [26]:
atom_indices_o1 = [a.GetIdx() for a in atoms1 if a.GetSymbol()=='O']

In [27]:
print(atom_indices_s1)
print(atom_indices_n1)
print(atom_indices_o1)

[12]
[11, 13]
[9, 10, 14, 15]


In [28]:
s = atom_indices_s1[0]
n1,n2 = atom_indices_n1
o1,o2 = atom_indices_o1[2:]

In [29]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s,n1) # N

1.6843525337217278

In [30]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s,n2) # N

1.6141500481907425

In [31]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s,o1) # O

1.4612819353715265

In [32]:
Chem.rdMolTransforms.GetBondLength(opt1_sage200_conf,s ,o2) # O

1.4605230793477002

In [33]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage200_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 63.47608125120574
N1 - S - O1: 113.42871751376049
N1 - S - O2: 111.46036387456668
O1 - S - O2: 116.0529900118958


In [34]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage210_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 98.99114386806883
N1 - S - O1: 116.55567527264776
N1 - S - O2: 109.78110720263066
O1 - S - O2: 122.71325775724155


In [35]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage220_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 103.25878214571215
N1 - S - O1: 111.73114257116715
N1 - S - O2: 106.93879395727107
O1 - S - O2: 123.81954086901293


In [36]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage221_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage221_conf,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage221_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt1_sage221_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 103.02876388391553
N1 - S - O1: 111.61228299733904
N1 - S - O2: 106.83915866217968
O1 - S - O2: 123.99766028136226


In [37]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,n1,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf1,o1,s,o2)) # O1 - S - O2

N1 - S - N2: 99.05909762234637
N1 - S - O1: 115.56915967238257
N1 - S - O2: 107.47423723768361
O1 - S - O2: 118.59078484711998


# Molecule 31, QCArchive ID 36972425

Bad with Sage 2.0 and 2.1, fixed with 2.2

Has many conformers--index 29-38. Some are distorted, some aren't

In [38]:
# test_mol2 = industry_benchmark_rm[31][1]
# test_mol2

In [39]:
# test_mol2.to_file('qcaid_36972425.sdf',file_format='sdf')

In [40]:
test_mol2 = Molecule.from_file('qcaid_36972425.sdf')

In [42]:
ie_sage200_qm2,me_sage200_qm,min_mol_sage200_qm2 = minimize_energy(test_mol2,ForceField('openff_unconstrained-2.0.0.offxml'))
u_sage200_2 = mda.Universe(min_mol_sage200_qm2.to_rdkit())
u_sage200_2.atoms.write("sage-200_qcaid_36972425_opt.pdb")

In [43]:
ie_sage210_qm2,me_sage210_qm2,min_mol_sage210_qm2 = minimize_energy(test_mol2,ForceField('openff_unconstrained-2.1.0.offxml'))
u_sage210_2 = mda.Universe(min_mol_sage210_qm2.to_rdkit())
u_sage210_2.atoms.write("sage-210_qcaid_36972425_opt.pdb")

In [45]:
ie_sage220_qm2,me_sage220_qm2,min_mol_sage220_qm2 = minimize_energy(test_mol2,ForceField('../../../openff_unconstrained-2.2.0.offxml',allow_cosmetic_attributes=True))
u_sage220_2 = mda.Universe(min_mol_sage220_qm2.to_rdkit())
u_sage220_2.atoms.write("sage-220_qcaid_36972425_opt.pdb")

In [46]:
ie_sage221_qm2,me_sage221_qm2,min_mol_sage221_qm2 = minimize_energy(test_mol2,ForceField('../../../openff_unconstrained-2.2.1-rc1.offxml',allow_cosmetic_attributes=True))
u_sage221_2 = mda.Universe(min_mol_sage221_qm2.to_rdkit())
u_sage221_2.atoms.write("sage-221_qcaid_36972425_opt.pdb")

In [69]:
u_qm_2 = mda.Universe(test_mol2.to_rdkit())
u_qm_2.atoms.write("qm_qcaid_36972425_opt.pdb")

In [47]:
min_mol_sage220_qm2.to_file('qcaid_36972425_optsage220.sdf',file_format='sdf')
min_mol_sage221_qm2.to_file('qcaid_36972425_optsage221.sdf',file_format='sdf')

In [48]:
opt2_sage200 = min_mol_sage200_qm2.to_rdkit()
opt2_sage200_conf = opt2_sage200.GetConformer()

In [49]:
opt2_sage210 = min_mol_sage210_qm2.to_rdkit()
opt2_sage210_conf = opt2_sage210.GetConformer()

In [50]:
opt2_sage220 = min_mol_sage220_qm2.to_rdkit()
opt2_sage220_conf = opt2_sage220.GetConformer()

In [51]:
opt2_sage221 = min_mol_sage221_qm2.to_rdkit()
opt2_sage221_conf = opt2_sage221.GetConformer()

In [52]:
qm_rdkit2 = test_mol2.to_rdkit()
qm_conf2 = qm_rdkit2.GetConformer()

### Geometries

In [53]:
atoms2=[a for a in opt2_sage200.GetAtoms()]

In [54]:
atom_indices_s2 = [a.GetIdx() for a in atoms2 if a.GetSymbol() == 'S']
atom_indices_n2 = [a.GetIdx() for a in atoms2 if a.GetSymbol()=='N']
atom_indices_o2 = [a.GetIdx() for a in atoms2 if a.GetSymbol()=='O']

In [55]:
print(atom_indices_s2)
print(atom_indices_n2)
print(atom_indices_o2)

[13]
[12, 16]
[4, 14, 15]


In [56]:
s = atom_indices_s2[0]
n1,n2 = atom_indices_n2
o1,o2 = atom_indices_o2[1:]

In [57]:
Chem.rdMolTransforms.GetBondLength(opt2_sage200_conf,s,n1) # N

1.6647789716184596

In [58]:
Chem.rdMolTransforms.GetBondLength(opt2_sage200_conf,s,n2) # N

1.6142719025528554

In [59]:
Chem.rdMolTransforms.GetBondLength(opt2_sage200_conf,s,o1) # O

1.459182943800153

In [60]:
Chem.rdMolTransforms.GetBondLength(opt2_sage200_conf,s ,o2) # O

1.4601189005964996

In [61]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage200_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage200_conf,n1,s,o1)) # N1 - S - O1
print('N2 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage200_conf,n2,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage200_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage200_conf,o1,s,o2)) # O1 - S - O2


N1 - S - N2: 72.0450096074957
N1 - S - O1: 115.37159939182366
N2 - S - O1: 79.1833259591064
N1 - S - O2: 114.2494046319114
O1 - S - O2: 115.78921157369467


In [62]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage210_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage210_conf,n1,s,o1)) # N1 - S - O1
print('N2 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage210_conf,n2,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage210_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage210_conf,o1,s,o2)) # O1 - S - O2



N1 - S - N2: 77.85793010230469
N1 - S - O1: 115.38856003526233
N2 - S - O1: 83.32978080305587
N1 - S - O2: 114.57483123650795
O1 - S - O2: 124.07650805400796


In [63]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage220_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage220_conf,n1,s,o1)) # N1 - S - O1
print('N2 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage220_conf,n2,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage220_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage220_conf,o1,s,o2)) # O1 - S - O2

N1 - S - N2: 103.35612593977908
N1 - S - O1: 108.4346294918543
N2 - S - O1: 102.6171321476026
N1 - S - O2: 110.62904723347857
O1 - S - O2: 126.03112454987792


In [65]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage221_conf,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage221_conf,n1,s,o1)) # N1 - S - O1
print('N2 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage221_conf,n2,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage221_conf,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(opt2_sage221_conf,o1,s,o2)) # O1 - S - O2

N1 - S - N2: 103.79500523605871
N1 - S - O1: 108.35736932709855
N2 - S - O1: 102.62497073634181
N1 - S - O2: 110.54361207027118
O1 - S - O2: 125.95797254723863


In [64]:
print('N1 - S - N2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf2,n1,s,n2)) # N1 - S - N2
print('N1 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(qm_conf2,n1,s,o1)) # N1 - S - O1
print('N2 - S - O1:',Chem.rdMolTransforms.GetAngleDeg(qm_conf2,n2,s,o1)) # N1 - S - O1
print('N1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf2,n1,s,o2)) # N1 - S - O2
print('O1 - S - O2:',Chem.rdMolTransforms.GetAngleDeg(qm_conf2,o1,s,o2)) # O1 - S - O2

N1 - S - N2: 110.18671827912529
N1 - S - O1: 105.13824121121552
N2 - S - O1: 105.55160323063248
N1 - S - O2: 106.29639112707902
O1 - S - O2: 124.68927907251683
